In [23]:
#Importing Packages
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd


In [24]:
url = 'https://www.myntra.com/dresses?f=Gender%3Amen%20women%2Cwomen&sort=popularity'
driver = webdriver.Chrome()
driver.get(url)

In [5]:
title = driver.title
print(title)

Dresses Online - Low Price Offer on Dresses for Women at Myntra


In [31]:
def extract_product_info(li_element):
    try:
        wait = WebDriverWait(li_element, 10)

        # Wait for the product meta info to load
        product_meta_info = wait.until(
            EC.presence_of_element_located((By.CLASS_NAME, "product-productMetaInfo"))
        )

        product_id = li_element.get_attribute("id")

        product_name = li_element.find_element(By.CLASS_NAME, "product-product").text

        brand = li_element.find_element(By.CLASS_NAME, "product-brand").text

        try: 
            discounted_price = li_element.find_element(By.CLASS_NAME, "product-discountedPrice").text
            price = li_element.find_element(By.CLASS_NAME, "product-strike").text

        except:
            discounted_price = "NA"
            price = li_element.find_element(By.CLASS_NAME, "product-price").text

        image_element = li_element.find_element(By.TAG_NAME, "img")

        image_url = image_element.get_attribute("src")

        product_link = li_element.find_element(By.TAG_NAME, "a").get_attribute("href")

        return {
            "id": product_id,
            "product_name": product_name,
            "brand": brand,
            "price": price,
            "discounted_price": discounted_price,
            "image_url": image_url,
            "product_link": product_link,
        }

    except Exception as e:
        print(f"Error extracting product info from <li>: {e}")
        return None


In [32]:
li_container = driver.find_element(By.CLASS_NAME, "results-base")

li_elements = li_container.find_elements(By.TAG_NAME, 'li')

products = []
count = 0

for li in li_elements:
    if count > 10:
        break
    
    product = extract_product_info(li)
    if product:
        products.append(product)
    count = count + 1

print(count)
df = pd.DataFrame(products)

# Save the DataFrame to a CSV file
df.to_csv('products.csv', index=False)
print("Data saved to 'products.csv'")

11
Data saved to 'products.csv'
